<a href="https://colab.research.google.com/github/gorogoro-uk/TensorFlow/blob/master/Tensorflow_Model_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TensorFlow Model Summary**

### Callbacks

In [ ]:
# Accuracy Callback
# check accuracy at end of every epoch and stop training once reached desired level

ACCURACY_LIMIT = 0.99
# define accuracy callback class
class myAccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > ACCURACY_LIMIT):
            print(f"\nreached {ACCURACY_LIMIT*100:.3%} accuracy, terminating training.\n")
            self.model.stop_training = True

# instantiate callback object
acc_callback = myAccuracyCallback()

# reference callback objcet in model.fit()
mnist_model.fit(
    x_train,
    y_train,
    epochs=10,
    callbacks=[acc_callback]
)

In [ ]:
# Learning Rate Callback
# use different learning rate every epoch to find optimal value
# plot loss v learning to find lowest stable value


# instantiate a LearningRateScheduler
# make learning a function of epoch 
# 1e-8 < learning_rate < 1e-4 (when epoch = 100)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# reference learning rate scheduler in model.fit()
history = model.fit(dataset, 
                    epochs=100, 
                    callbacks=[lr_schedule])


### Model Types

In [ ]:
# 1. Simple Dense Neural Net

# input: [batch, width(28), height(28), channel(1)]
mnist_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # [batch, width x height x channel(784)] flatten image matrix
    tf.keras.layers.Dense(500, activation='relu'),  # [batch, dnn_units(500)]
    tf.keras.layers.Dense(1, activation='sigmoid')  # [batch, dnn_units(1)]
])

from tensorflow.keras.optimizers import RMSprop

mnist_model.compile(optimizer=RMSprop(lr=0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy']
)

In [ ]:
# 2. Convolutional Neural Net for Image Classification

# 2D convolutions to extract features
# MaxPooling to downsample and reduce size
sign_model = tf.keras.models.Sequential([                                           # [batch, width(28), height(28), channel(1)]
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),  # [batch, filter scans(26), filter scans(26), filters(64)]
    tf.keras.layers.MaxPooling2D(2,2),                                              # [batch, 15-3+1(13), 13, 64]
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),                         # [batch, filter scans(11), filter scans(11), filters(128)]
    tf.keras.layers.MaxPooling2D(2, 2),                                             # [batch, 11-3+1(5), 5, 128]
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),                         # [batch, filter scans(3), filter scans(3), filters(256)]
    tf.keras.layers.MaxPooling2D(2, 2),                                             # [batch, 3-3+1(1), 1, 256]
    tf.keras.layers.Flatten(),                                                      # [batch, 1x1x256(256)]
    tf.keras.layers.Dense(128, activation='relu'),                                  # [batch, dnn_units(128)]
    tf.keras.layers.Dense(26, activation='softmax')                                 # [batch, dnn_units(26)]
])

sign_model.compile(optimizer='Adam', 
                   loss='sparse_categorical_crossentropy', 
                   metrics=['accuracy'])

In [ ]:
# 3. NLP Word Embeddings

bbc_model = tf.keras.Sequential([                                                     # [batch, sentence_length]
    tf.keras.layers.Embedding(vocab_size, embed_dim, input_length=sentence_length),   # [batch, sentence_length, embed_dimension]
    tf.keras.layers.GlobalAveragePooling1D(),                                         # [batch, embed_dimension]     avg embed_val per sentence
    tf.keras.layers.Dense(24, activation='relu'),                                     # [batch, dnn_units(24)]
    tf.keras.layers.Dense(6, activation='softmax')                                    # [batch, dnn_units(6)]
])

bbc_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

In [ ]:
# 4. NLP Transfer Learning with Pretrained Word Embeddings

# embed_matrix: [vocab, embed_dimension]
glove_model = tf.keras.Sequential([                         # [batch, sentence_length]
    tf.keras.layers.Embedding(vocab, 
                              embed_dimension, 
                              input_length=sentence_length, 
                              weights=[embed_matrix], 
                              trainable=False),             # [batch, sentence_length, embed_dimension]
    tf.keras.layers.Dropout(0.2),                           # [batch, sentence_length, embed_dimension]
    tf.keras.layers.Conv1D(64, 5, activation='relu'),       # [batch, filter scans(12), filters(64)]
    tf.keras.layers.MaxPooling1D(pool_size=4),              # [batch, filter scans(3), filters(64)]  stride=pool_size=4
    tf.keras.layers.LSTM(64),                               # [batch, lstm_units(64)]
    tf.keras.layers.Dense(1, activation='sigmoid')          # [batch, dnn_units(1)]
])

glove_model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
# 5. NLP LSTM with Regularisation

# embed_matrix: [total_words, sequence_length]
shakespeare_model = Sequential(                                                                # [batch, sentence_length(10)]
    tf.keras.layers.Embedding(total_words, 100, input_length=10),                              # [batch, sentence_length(10), embed_dimension(100)]
    tf.keras.layers.Bidirectional(LSTM(150, return_sequences = True)),                         # [batch, sentence_length(10), lstm_unitsx2(300)]
    tf.keras.layers.Dropout(0.2),                                                              # [batch, 10, 300]
    tf.keras.layers.LSTM(100),                                                                 # [batch, lstm_units(100)]
    tf.keras.layers.Dense(1605, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  # [batch, dnn_units(1605)]
    tf.keras.layers.Dense(3211, activation='softmax')                                          # [batch, dnn_units(3211)]
)

shakespeare_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


### Regularisation

**Image Augmentation**

Create additional training data with more variation to reduce overfitting. Use existing training images as basis for new images by manipulating them with shifts, zooms, shears, rotations, flips.

*   Rotate
*   Shift
*   Shear
*   Zoom
*   Flip

Example:
train_image_datagen = ImageDataGenerator(rescale=1/255.,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

**Drop Out Layer**

Assign a zero value to a random selection of weights in DNN layer.

Example:
tf.keras.layers.Dropout(0.2)


**L1, L2 Regularisation on Dense Layer**

Apply regularisation factor in loss function for a given layer of DNN.

L1: absolute value of weights: some weights can be zero, many solutions, better for less complex functions

L2: squared value of weights: weights all positive, but small, only one solution, can fit a complex function

Example:
Dense(num_units, activation='relu', kernel_regularizer=regularizers.l2(0.01)
